In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import torch as pt
import time
import sys

sys.path.insert(0, '..')

from NNsolver import NNSolver
from NNarchitectures import DenseNet_g
from problems import DoubleWell
from utilities import get_X_process, plot_NN_evaluation, compute_PDE_loss

%load_ext autoreload
%autoreload 2

device = pt.device('cuda')

In [ ]:
T = 0.3
problem = DoubleWell(d=20, d_1=20, d_2=0, T=0.3, eta=0.5, kappa=0.5, diagonal=False)
problem.compute_reference_solution()
problem.compute_reference_solution_2()
K = 2000
K_batch = 2000
print_every = 500
delta_t = 0.01
sq_delta_t = pt.sqrt(pt.tensor(delta_t))
N = int(np.ceil(T / delta_t))
gradient_steps = (N - 1) * [3000] + [30000]
learning_rates = [0.0007] * (N - 1) + [0.001]

model = NNSolver(problem, 'DoubleWell', learning_rates=learning_rates, gradient_steps=gradient_steps, NN_class=DenseNet_g, K=K, 
                 K_batch=K_batch, delta_t=delta_t, print_every=print_every, method='implicit')

model.Y_n = [DenseNet_g(problem.d, 1, lr=learning_rates[n], problem=problem, arch=[20, 20, 20, 20]).to(device) for n in range(N)] + [problem.g]

In [ ]:
model.train()

In [ ]:
fig = plot_NN_evaluation(model, n=N-2, reference_solution=False, Y_0_true=34.26871278084)

### compute reference value for V(x_0, 0)

In [ ]:
problem.modus = 'np'

expectation = 0
K = 1000000
L = 10
for l in range(L):
    X, xi = get_X_process(problem, K, delta_t, seed=l, t=0, x=problem.X_0)
    expectation += np.sum(np.exp(-problem.g(X[N, :, :]))) / (L * K)

In [ ]:
-np.log(expectation)

### compute reference loss

In [ ]:
K = 1000
Y_ref = np.zeros([N + 1, K])
Y_pred = np.zeros([N + 1, K])

X, xi = get_X_process(problem, K, delta_t, seed=44, t=0, x=problem.X_0)

for n in range(N + 1):
    for k in range(K):
        problem.modus = 'np'
        X_, xi = get_X_process(problem, 10000, delta_t, seed=44, t=n * delta_t, x=X[n, k, :])
        Y_ref[n, k] = -np.log(np.mean(np.exp(-problem.g(X[-1, :, :]))))

    problem.modus = 'pt'   
    Y_pred[n, :] = model.Y_n[n](pt.tensor(X[n, :, :]).float().to(device)).cpu().detach().squeeze().numpy()
    problem.modus = 'np'   
    
np.mean(np.abs((Y_pred - Y_ref) / Y_ref))

In [ ]:
np.mean(np.abs((Y_pred - Y_ref) / Y_ref))

In [ ]:
problem.modus = 'pt'
pde_loss = compute_PDE_loss(problem, delta_t=0.01, K=100, Y_n=model.Y_n, vfun=None, testOde=None, seed=44, 
                            print_every=1)

In [ ]:
np.mean(pde_loss[:-1])